In [1]:
gap('LoadPackage("numericalsgps")')

true

In [2]:
def SNG_SpecialGaps(SNG):
    try:
        SGs = gap(f"SpecialGaps({SNG})")
    except:
        SGs = []
    NSs = []
    for k in SGs:
        NSs.append(gap(f"AddSpecialGapOfAffineSemigroup({k},{SNG})"))
    return NSs

In [3]:
Nd = gap('''AffineSemigroup( [ [ 0, 0, 1 ], [ 0, 1, 0 ], [ 1, 0, 0 ], 
[ 1, 0, 1 ], [ 1, 1, 0 ], [ 2, 0, 0 ], [ 3, 0, 0 ] ] )''')
def oversemigroup_function(SNG):
    HashMap = {}
    def generate_oversemigroup_tree(N):
        nonlocal HashMap
        global Nd
        if isinstance(N,list):
            for Nm in N: 
                OverN = SNG_SpecialGaps(Nm)
                if f"{Nd}" in [str(A) for A in OverN]:
                    HashMap[gap(f"Gaps({Nm})")] = [Nd]
                else:
                    if str(Nm) == f"{Nd}":
                        HashMap[Nd] = []
                    else:
                        HashMap[gap(f"Gaps({Nm})")] = [gap(f"Gaps({M})") for M in OverN]
                generate_oversemigroup_tree(OverN)
        else:
            OverN = SNG_SpecialGaps(N)
            HashMap[gap(f"Gaps({N})")] = [gap(f"Gaps({M})") for M in OverN]
            generate_oversemigroup_tree(OverN)

    generate_oversemigroup_tree(SNG)
    return HashMap

In [10]:
SNG_1 = gap("AffineSemigroupByGaps([[1,0,0],[0,1,0],[0,0,1]])")
D = DiGraph(oversemigroup_function(SNG_1))
P = Poset(D)
SNG_1_plot = P.plot(vertex_shape="",vertex_size=1000,figsize=12,edge_style=":", edge_thickness = 0.1)
SNG_1_plot.save("Diagrama_Hasse_SNG_N3.png")

In [6]:
import itertools
import copy

def SNG_I_decomposition(SNG):
    SGs = gap(f"SpecialGaps({SNG})")
    SNG_2 = SNG
    I = []
    C = [SNG]
    while C != []:
        B = []
        for SNG in C:
            B.extend(SNG_SpecialGaps(SNG))
        B.sort()
        B = list(k for k,_ in itertools.groupby(B))
        B = [S for S in B if not specialgaps_in_SNG(S,SGs)]
        B = [S for S in B if not exist_subset(S,I)]
        C = [S for S in B if not EsIrreducible(S)]
        I.extend([S for S in B if S not in C])
    C_S = {S : Ev_C_S(S,SGs) for S in I}
    Descomp = Opt_S(SNG_2,I,C_S)
    return Descomp

def specialgaps_in_SNG(SNG,SpecialGaps):
    for N in SpecialGaps:
        if not gap(f"{N} in {SNG}"):
            return False
    return True

def exist_subset(SNG,Conjunto_SNGs):
    Gaps_SNG = gap(f"Gaps({SNG})")
    try:
        Conjunto_SNGs.remove(SNG)
    except:
        pass
    for N in Conjunto_SNGs:
        Gaps_N = gap(f"Gaps({N})")
        if A_is_subset(Gaps_SNG,Gaps_N):
            return True
    return False

def A_is_subset(setA,setB):
    for N in setA:
        if N not in setB:
            return False
    return True

def EsIrreducible(SNG):
    return len(gap(f"SpecialGaps({SNG})"))==1

def Ev_C_S(SNG, SpecialGaps):
    return [e for e in SpecialGaps if not gap(f"{e} in {SNG}")]

def Opt_S(SNG,I,CS):
    I2 = []
    for K in I:
        I3 = copy.deepcopy(I)
        if not exist_subset(K,I3):
            I2.append(K)
    if len(I) != len(I2):
        I = I2
        for k in [S for S in I if S not in I2]:
            del CS[k]
    AB = [list(comb) for i in range(2, len(I)+1) for comb in itertools.combinations(CS.values(), i)]
    AB2 = [list(comb) for i in range(2, len(I)+1) for comb in itertools.combinations(I, i)]
    Gaps_SNG = gap(f"Gaps({SNG})")
    SGs = gap(f"SpecialGaps({SNG})")
    i=0
    Combos = []
    for recurso in AB:
        prim = recurso[0]
        for N in recurso[1:]:
            prim.extend(N)
        prim.sort()
        prim = list(k for k,_ in itertools.groupby(prim))
        if gap(f"{prim} = {SGs}"):
            gaps_sets = [list(gap(f"Gaps({S})")) for S in AB2[i]]
            gaps_sets_0 = list(gaps_sets[0])
            for N in gaps_sets[1:]:
                gaps_sets_0.extend(list(N))
            gaps_sets_0.sort()
            gaps_sets_0 = list(k for k,_ in itertools.groupby(gaps_sets_0))
            if gap(f"{gaps_sets_0} = {Gaps_SNG}"):
                Combos.append(gaps_sets)
                break
        i=i+1
    return Combos

In [7]:
Test_1 = gap("AffineSemigroupByGaps([[0, 1], [1, 0], [1, 1], [2, 0], [2, 1]])")
SNG_I_decomposition(Test_1)

[[[[ 1, 0 ], [ 2, 0 ], [ 2, 1 ]], [[ 0, 1 ], [ 1, 1 ], [ 2, 1 ]]]]

In [8]:
Test_2 = gap("AsAffineSemigroup(NumericalSemigroup(5,21,24,28,32))")
Combos = SNG_I_decomposition(Test_2)

In [9]:
def flatten_chain(matrix):
    return list(itertools.chain.from_iterable(matrix))

for k in Combos[0]:
    print(gap(f"MinimalGenerators(NumericalSemigroupByGaps({list(flatten_chain(k))}))"))

[ 5, 8 ]
[ 5, 7 ]
